# Tensorflow Serving - Advanced Usecases

TFS provides some advanced features, that helps models in software engineerging perspective to scale on large organizations and obviously model serving perspective. For example basic deployment of models requires the developer to provide the model location (either mounted or copied to local storage) and then serve. But instead of that we can provide a remote location to model storage and then point that to the TFS server. Since TFS polls the storage to see any changes this method helps well to decouple the devops and ML model development process.

<center><image src="imgs/4.jpg" width="500"/></center>

If our remote storage is publically accessible the configuration is easy. But if they are private then we need to provide additional parameters when deploying the server for the first time. Based on where we are using as the remote storage, it is better to use the documentation to see exact details.

Also in today world, models are very large. Therefore it is impractical to use them in many scenarios directly due to computational cost, resource cost etc. To mitigate such issues we can optimize our models using few optimization techniques. Below are some brief explanations of them.


### Quantization

Model quantization allows to reduce the computational complexity by reducing the precision of the weight representation. Generally models store numbers as 32bit floats with 1 bit to store the sign, 8 bits to exponent and rest to the floating number. But model weights can instead be expressed in 16bit floating numbers or 8bit ints. This way we can represent weights with fewer bytes and therefore less memory usage. Also due to that we can have faster inference times.

But as we can see, quantization reduce the model precision. Therefore it is important to check the model performance after quantization process.


### Pruning

Instead of reducing model precision, we can reduce the network size by removing unnecessary weights. In practice this means setting values to zero. This helps to increased model compression ratios due to sparse representations.


### Distillation


Model distillation introduce a new technique to reduce model size. Basically idea in distillation is to train a smaller, less complex network to learn trained tasks from a more complex model. By using the complex model as the teacher smaller model can be forced to learn an objective with few weights and complexity.


<center><image src="imgs/5.jpg" width="500"/></center>

</br>

> Theres a library named `TensorflowRT` from nvidia to perform quantization optimizations to models. Check that out for more info!


### TFLite

TFLite perform optimizations to models and convert them to smaller sizes for deployments to mobile/ IoT devices. But these models can also directly be served using TFS and can provide low latency inference with lower resource cost. Another advantage is we are not bound to Nvidia GPUs.

But as always there are caveats such as some tensorflow operations may not be converted to TFLite, limited serving support etc.

To convert a model to TFLite, we need to first start with a exported model in SavedModel format. Below is an example optimization using TFLite.

In [ ]:
import tensorflow as tf


model_dir = 'data/tfx/Trainer/model/6/format-Serving/'

convertor = tf.lite.TFLiteConverter.from_saved_model(model_dir)

convertor.optimizations = [
    tf.lite.Optimize.DEFAULT
]

convertor.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = convertor.convert()

with open('data/tflite/model.tflite', 'wb') as f:
    f.write(tflite_model)

> TFLite provides 3 main model optimization objectives. OPTIMIZE_FOR_LATENCY, OPTIMIZE_FOR_SIZE and DEFAULT. DEFAULT is the middle ground of other 2 values. We can add those objectives along with other converting parameters to the TFLiteConverter.

As TFLite does not support all the tensorflow operations, sometimes model conversion may fail. In such cases we can try out adding additional tensorflow operations support to the convertor configuration. But this will increase the converted model size by about 30mb.

But if it still did not work, then sorry! May be you can ask from tf community to help!

### Monitoring the TFS instance

In production systems it is important to see what happens and logging events plays a major role. To help with such cases TFS provide endpoints to matrices which can be consumed by logging applications like prometheus. (Prometheus is a logging application closely related wih kubernetes eco system)

To track inference matrices, we need to run the TFS and prometheus side by side. They will communicate via rest apis and provide necessary facilities for monitoring. Setting up prometheus require us to provide a config to prometheus server and adding some additional configs to TFS deployment as well.

